<a href="https://colab.research.google.com/github/sepidehnaghshineh/LD/blob/main/Downloading_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading the data for Lesion Detection task

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
cd ./drive/MyDrive/PhD_Project/

/content/drive/MyDrive/PhD_Project


In [ ]:
!apt-get install wget
!pip install tqdm

In [ ]:
import subprocess

username = "sepidehkani"
password = "ketsum-dydcir-5Mepqu"
url = "https://physionet.org/files/vindr-mammo/1.0.0/"  # Test the base URL

command = f"wget --user={username} --password={password} {url}"

try:
    result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
    print("wget successful")
    print(result.stdout)
    print(result.stderr)

except subprocess.CalledProcessError as e:
    print("wget failed")
    print(e.stderr)

In [ ]:
import subprocess

username = "sepidehkani"
password = "ketsum-dydcir-5Mepqu"
url = "https://physionet.org/files/vindr-mammo/1.0.0/images/"

command = f"wget -q -O- --user={username} --password={password} {url}"

try:
    html_content = subprocess.check_output(command, shell=True, text=True)
    print(html_content)  # Print the HTML content
except subprocess.CalledProcessError as e:
    print(e)

In [ ]:
import os
import subprocess
import math
import json
import re
import time

def create_download_chunks(base_url, username, password, output_dir="images", chunk_size_gb=5, chunk_manifest_file="download_chunks.json", max_retries=3, retry_delay=5):
    """Creates a manifest file with download chunks for VinDr-Mammo dataset with retry."""
    base_url_images = f"{base_url}{output_dir}/"

    for retry in range(max_retries):
        try:
            command = f"wget -q -O- --user={username} --password={password} {base_url_images}"
            output = subprocess.check_output(command, shell=True, text=True)

            # Corrected regex to extract study IDs
            study_ids = re.findall(r'<a href="([0-9a-f]{32})/">', output)

            if not study_ids:
                if retry < max_retries - 1:
                    print(f"Retry {retry + 1}/{max_retries}: No study IDs found. Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                    continue
                else:
                    print("Error: No study IDs found after multiple retries. Check credentials and URL.")
                    return

            num_studies = len(study_ids)
            studies_per_chunk = math.ceil((chunk_size_gb * 1024 * 1024 * 1024) / (4 * 100 * 1024))
            num_chunks = math.ceil(num_studies / studies_per_chunk)

            chunks = []
            for chunk_num in range(num_chunks):
                start_index = chunk_num * studies_per_chunk
                end_index = min((chunk_num + 1) * studies_per_chunk, num_studies)
                chunk_studies = study_ids[start_index:end_index]
                chunks.append(chunk_studies)

            with open(chunk_manifest_file, "w") as f:
                json.dump(chunks, f, indent=4)

            print(f"Chunk manifest created: {chunk_manifest_file}")
            print(f"Total chunks: {num_chunks}")
            return #Exit the function if successful.

        except subprocess.CalledProcessError as e:
            if retry < max_retries - 1:
                print(f"Retry {retry + 1}/{max_retries}: Error retrieving study IDs: {e}. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                continue
            else:
                print(f"Error retrieving study IDs after multiple retries: {e}")
                return
        except Exception as e:
            if retry < max_retries - 1:
                print(f"Retry {retry + 1}/{max_retries}: An unexpected error occurred: {e}. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                continue
            else:
                print(f"An unexpected error occurred after multiple retries: {e}")
                return

def download_chunk(base_url, username, password, chunk_manifest_file, chunk_index):
    # ... (rest of the download_chunk function remains the same)
    with open(chunk_manifest_file, "r") as f:
        chunks = json.load(f)

    if chunk_index < 0 or chunk_index >= len(chunks):
        print("Invalid chunk index.")
        return

    chunk_studies = chunks[chunk_index]
    base_url_images = f"{base_url}images/"

    print(f"Downloading chunk {chunk_index + 1} of {len(chunks)}...")

    for study_id in chunk_studies:
        study_url = f"{base_url_images}{study_id}/"
        print(f"  Downloading study {study_id}...")
        command = f"wget -r -N -c -np --user={username} --password={password} {study_url}"

        try:
            subprocess.run(command, shell=True, check=True)
            print(f"  Study {study_id} downloaded successfully.")
        except subprocess.CalledProcessError as e:
            print(f"  Error downloading study {study_id}: {e}")
        except Exception as e:
            print(f"Unexpected error: {e}")

    print(f"Chunk {chunk_index + 1} downloaded.")





In [ ]:
# Example usage:
base_url = "https://physionet.org/files/vindr-mammo/1.0.0/"
username = "sepidehkani"
password = "ketsum-dydcir-5Mepqu"
chunk_manifest_file = "download_chunks.json"

# Create the chunk manifest (run this once)
create_download_chunks(base_url, username, password, chunk_manifest_file=chunk_manifest_file)

# Download each chunk in separate code blocks:
# Example: Download chunk 0
download_chunk(base_url, username, password, chunk_manifest_file, 0)

